<a href="https://colab.research.google.com/github/Mihail-Chr/projects/blob/main/ML/car_cost/ML_car_cost_regression_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении имеются исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости. Метрика  RMSE должна быть не ниже 2500

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [ ]:
! pip install catboost
! pip install phik

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns


# import phik
from phik.report import plot_correlation_matrix
from phik import phik_matrix

import sklearn.utils
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer,IterativeImputer,SimpleImputer
from sklearn.metrics import (confusion_matrix,mean_squared_error,precision_recall_curve,roc_auc_score,classification_report,
                             r2_score,accuracy_score,make_scorer,f1_score,roc_curve,RocCurveDisplay,
                             mean_absolute_error,recall_score,precision_score,ConfusionMatrixDisplay)

from sklearn.feature_selection import SelectKBest,f_classif, mutual_info_classif
from sklearn.inspection import permutation_importance

from sklearn.model_selection import (train_test_split,GridSearchCV,cross_val_score)

from sklearn.metrics import root_mean_squared_error,mean_absolute_error

from sklearn.model_selection import (train_test_split,GridSearchCV,cross_val_score)
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,LogisticRegression,Ridge, PassiveAggressiveRegressor,SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import Pipeline

from catboost import CatBoostRegressor,CatBoostClassifier,Pool, cv
import lightgbm as lgb
from lightgbm import LGBMClassifier,LGBMRegressor


from numba import jit, cuda, njit


import warnings
from sklearn.exceptions import ConvergenceWarning
ConvergenceWarning('ignore')
import sys
import os
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = ('ignore::UserWarning,ignore::ConvergenceWarning,ignore::RuntimeWarning')

#optuna.logging.set_verbosity(optuna.logging.WARNING)

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.2f}'.format
pd.DataFrame.iteritems = pd.DataFrame.items
RANDOM_STATE = 255
TEST_SIZE = 0.25
warnings.filterwarnings('ignore')
#pd.options.mode.copy_on_write = True

In [ ]:
# функция преобразования в bool

def bul(dat,col,tru,nt):
    dat[col][dat[col]==tru]=1
    dat[col][dat[col]==nt]=0
    #dat[col]=dat[col].astype('boolean')
    #voc_k[col]=(nt,tru)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Обработка данных

In [ ]:
avto_df = pd.read_csv('/content/drive/MyDrive/data/autos.csv')
avto_df.info()
display(avto_df.head())

# удаляем столбцы с "технической информацией"
del_col = ['DateCrawled','DateCreated','NumberOfPictures','PostalCode','LastSeen']
avto_df = avto_df.drop(del_col, axis=1).copy()

# удаляем дубликаты
avto_df = avto_df.drop_duplicates()

display(avto_df.describe(include = "all"))

avto_df1 = avto_df.copy()
avto_df.hist(figsize=(20, 5), layout = (-1, 5))
plt.show()

### Первичные выводы:
 - наблюдается аномалии в годе выпуска, мощности
 - месяцев 13, вместо 12
 - нужно исследовать авто , стоиостью около 0
 - нужно исследовать авто , мощностью около 0
 - исследовать значения года больше текущего
 - исследовать мощность более "реальной" (650 л.с.)

In [ ]:
#произведем отчистку от нереалистичных данных

# удалим авто с RegistrationYear <1986 и > 2016
avto_df = avto_df1.copy()
avto_df = avto_df[(avto_df['RegistrationYear']>1986) & (avto_df['RegistrationYear']<2016)].copy()

# удалим авто с мощность <50 и >650
avto_df = avto_df[(avto_df['Power']>10) & (avto_df['Power']<650)].copy()

# удалим авто с ценой  <50
avto_df = avto_df[(avto_df['Price']>100)].copy()

# объединим признак "по газу"
avto_df = avto_df.replace('lpg', 'gasoline').copy()
avto_df = avto_df.replace('cng', 'gasoline').copy()

display('количество пустых ячеек',avto_df.isnull().sum())
#avto_df.info()
#avto_df.head()

In [ ]:
# анализ категориальных признаков
for col in avto_df.columns:
    if avto_df[col].isnull().sum()!=0 :
        display(avto_df[col].value_counts(dropna=False))

### Предобработка данных и последующий анализ

In [ ]:
# по итогу анализа можно сделать вывод, что в некоторых оисательных свойствах присутствует категоря "other" , которая не столь информативна,
# поэтому можем значения NaN приравнять к ней. В случае же с Gearbox и Repaired применим inputer IterativeImputer

avto_df[['FuelType'б'VehicleType','Model']] = avto_df[['FuelType'б'VehicleType','Model']].fillna('other').copy()
#avto_df['VehicleType'] = avto_df['VehicleType'].fillna('other').copy()
#avto_df['Model'] = avto_df['Model'].fillna('other').copy()


bul(avto_df,'Gearbox','manual','auto')
bul(avto_df,'Repaired','yes','no')

imput = IterativeImputer(random_state=RANDOM_STATE)
avto_df['Gearbox'] = imput.fit_transform(avto_df[['Gearbox']])
avto_df['Repaired'] = imput.fit_transform(avto_df[['Repaired']]).copy()
avto_df['Gearbox'] = round(avto_df['Gearbox'])
avto_df['Repaired'] = round(avto_df['Repaired'])

avto_df = avto_df.drop_duplicates( keep='first').copy()
avto_df = avto_df.reset_index(drop=True)


avto_df.info()
avto_df.head()

### Визуализация данных

In [ ]:
# составим базу описывающую автомобиль.
cat_col = ['Brand','Model','FuelType','VehicleType','Gearbox','RegistrationMonth','Repaired']
num_col = ['Price','RegistrationYear','Power','Kilometer']

col_auto_df = ['Price','Brand','Model','RegistrationYear','RegistrationMonth',
           'FuelType','VehicleType','Gearbox','Power','Kilometer','Repaired']


In [ ]:
interval_cols = num_col
avto_df[cat_col] = avto_df[cat_col].astype('category')
# 'PHIK матрица'
display ('PHIK матрица ')

phik_overview = avto_df.phik_matrix( interval_cols=interval_cols,n_jobs=-1)
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Greens',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.3,
    figsize=(10,9))

In [ ]:
for i in cat_col:
    if avto_df[i].nunique()<20:
        sns.countplot(y=i, data=avto_df)
        plt.show()

for n in num_col:
    display(n)
    fig, (ax_box, ax_hist) = plt.subplots(2,sharex = True,gridspec_kw = {'height_ratios': (.20, .80)})
    sns.boxplot(x = avto_df[n], ax = ax_box)
    plt.hist(avto_df[n],bins=100)
    ax_box.set(xlabel = '')
    ax_hist.set(xlabel = n)
    ax_hist.set(ylabel = 'count')
    plt.show()




#### Выводы и план по обработке данных
- Model и Brand имеют корреляцию = 1, значит несут в себе "дублирующий" смысл. Поскольку у Brand меньше значения коэфициентов с другими признами - удаляяем его.
- заполнить все "пустые" значения типом "other" . Выбор определен тем, что танный "непонятный"("неопределенный") класс присутсвует в свойствах "FuelType" "VihicleType"
- установим границы по мощности (40-650) л.с.
- установим границы по цене (100)
- авто с FuelType = cng  lng - преобразовываем в gasoline
- удалим данные с месяцем = 0
- установим границы по году : раз выгрузка было в 2016 - удаляем года после 2016, и удаляем авто старше 30 лет
- удалим дубликаты

### Выводы
- предобработка прошла успешно, оставлены более реалистичные данные

## Обучение моделей

### Подготовка данных

In [ ]:
pd.DataFrame.iteritems = pd.DataFrame.items
cat_col = ['Model','FuelType','VehicleType','Gearbox','RegistrationMonth','Repaired']
#df_auto[cat_col]=df_auto[cat_col].astype('category')
X=avto_df.drop(['Price','Brand'],axis=1).copy()
y=avto_df['Price'].copy()
# разделение данных на тестовую и тренировочную выборки
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE)


features_train=X_train
target_train=y_train


data_pool = Pool(features_train,target_train, cat_features= cat_col)

In [ ]:
# фукция параметров кросвалидации моделей
def cv_model(clf,X_train,y_train):


    param_grid = {}
    if clf.__class__.__name__ == 'SVR' :
        param_grid = {'models__C':np.arange(0.1, 1.2, 0.4) }

    elif clf.__class__.__name__ == 'Ridge' :
        param_grid = {'models__alpha': np.arange(0.3, 1, 0.1)}
    elif clf.__class__.__name__ == 'LinearRegression' :
        X_train = X_train.toarray()
        X_test = X_test.toarray()

    else  : param_grid = {}

    GridCV = GridSearchCV(
        model = clf,#pipe_final,
        param_grid=param_grid,
        cv=3,
        scoring='neg_root_mean_squared_error',
        error_score='raise',
        verbose=False,
        n_jobs=-1
    )
    GridCV.fit(X_train, y_train)
    clf_descr = clf.__class__.__name__

    print()
    print(clf_descr)

    print(f"best parames :   {GridCV.best_params_}")

    results = GridCV.cv_results_
    best_index = GridCV.best_index_

    fit_time = results['mean_fit_time'][best_index]
    score_time = results['mean_score_time'][best_index]
    RMSE = results['mean_test_score'][best_index] * -1

    print(f"Fit time: {ridge_fit_time}, Predict time: {ridge_score_time}, Best RMSE: {ridge_RMSE}")

    №print(f"best score :   {-GridSCV.best_score_}")

    return (clf_descr, RMSE, fit_time,score_time)

In [ ]:
def cross_m (X_tr,y_train):
    results = []
    for clf in (
        (DummyRegressor())
        (LogisticRegression(C=0.01, max_iter=500)),
        (Ridge(max_iter=300,random_state=RANDOM_STATE)),#,solver="sparse_cg")),
        (KNeighborsRegressor(n_neighbors=80)),
        (LinearRegression(n_jobs=-1)),

        (RandomForestRegressor(warm_start=True,random_state=RANDOM_STATE)),
        (SVR(max_iter=300)), #,cache_size=8000,gamma='auto',probability=True,
        (SGDRegressor( alpha=1e-4, n_iter_no_change=3, early_stopping=True)),
        (PassiveAggressiveRegressor()),

        (CatBoostRegressor(iterations=1000, learning_rate=0.001, depth=6,
                            loss_function='RSME',
                            random_seed = RANDOM_STATE,
                            early_stopping_rounds = 400,
                            allow_writing_files=False,
                            verbose= False))
    ):
        print("=" * 80)


        results.append(cv_model(clf,X_tr,y_train) )
    return results

In [ ]:
results_model = cross_m(X_train,y_train)

In [ ]:
%time
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', Ridge(random_state=RANDOM_STATE,alpha=0.9, max_iter=300, tol=0.1))])

model_reg=pipe_final
model_reg.fit(X_train,y_train)

In [ ]:
# важность признаков
y_pred=model_reg.predict(X_train)

RSME_reg=np.sqrt(mean_squared_error(y_train,y_pred))
display (RSME_reg)


features = X_train.columns
importance_values = model_reg['models'].coef_[0]
plt.barh(y=range(len(features)),
         width=importance_values,
         tick_label=features)
plt.show()

#report = classification_report(y_train, y_pred)
#print(report)

### построение модели Catboost с наилучшими параметрами

In [ ]:
# Catboost валидация

df_auto[cat_col]=df_auto[cat_col].astype('category')
X_train=df_auto.drop('Price',axis=1)
y_train=df_auto['Price']
data_pool = Pool(features_train,target_train, cat_features= cat_col)
model = CatBoostRegressor(loss_function='RMSE',
                          verbose=False,
                          subsample=0.6,
                          iterations=300) #,iterations=400

param_grid = {'learning_rate': np.arange(0.05, 0.063,0.002),
        'depth': range(1, 10),
        #'l2_leaf_reg': range(1,2),

        'min_data_in_leaf':range(1,5)
       }

grid_search_result = model.grid_search(param_grid,
                                       data_pool,
                                       early_stopping_rounds=10,
                                      )

display (grid_search_result['params'])
display(model.get_best_score())

In [ ]:
display (grid_search_result['params'])

feature_importance=model.get_feature_importance(prettified=True)
display(feature_importance)
plt.figure(figsize=(12, 6))
ax = sns.barplot(data=feature_importance,x=feature_importance['Importances'],
                 y=feature_importance['Feature Id'])
ax.set_title('Важность признаков', fontsize=16)
plt.show()
print(model.get_best_score())



In [ ]:
%time
model_best_cat =model
model_best_cat.fit(data_pool)

### построение модели LGBM с наилучшими параметрами

In [ ]:
model = LGBMRegressor(metric='neg_root_mean_squared_error',
                      random_state=RANDOM_STATE,
                      verbose= 0)

cv = StratifiedKFold(n_splits=5)
#cv = RepeatedStratifiedKFold(n_splits=10000, n_repeats=3, random_state=1)
model.fit(X_train, y_train,categorical_feature=cat_col)
n_scores = cross_val_score(model, X_train, y_train,
                           scoring='neg_root_mean_squared_error',
                           cv=cv, n_jobs=-1, error_score='raise')



print('RMSE:',(-max(n_scores)))

print('наилучшие параметры',model.get_params())


In [ ]:
best_model_LGBM =model#(model.best_estimator_)

# Plot feature importances
features = X_train.columns
importance_values = best_model_LGBM.feature_importances_
plt.barh(y=range(len(features)),
         width=importance_values,
         tick_label=features)
plt.show()



In [ ]:
%time


best_model_LGBM.fit(X_train, y_train)
%time

In [ ]:
# лучшая модель по метрике оказалась LGBM
%time
RSME_cat=np.sqrt(mean_squared_error(y_test, model_best_cat.predict(X_test)))
display (RSME_cat)


## Анализ моделей

- время выполнения лучше у Ridge 5.48 милсек
- целевое качество метрики RMSE достигнуто на всех моделях, у catboost лучшая =1519.56
- рекомендуемые модели для обучения Catboost и LGBM
- разные модели показали не одинаковую значимость признаков,
  но "тройка лидеров" у всех одна- ( "RegistrationYear", "Power", "Вrand")

<div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b>

Есть еще один вариант измерить время обучения и предсказания моделей, сделав это прямо в кросс-валидации:
    
```
results_ridge = gs_ridge.cv_results_
best_index_ridge = gs_ridge.best_index_

ridge_fit_time = results_ridge['mean_fit_time'][best_index_ridge]
ridge_score_time = results_ridge['mean_score_time'][best_index_ridge]
ridge_RMSE = results_ridge['mean_test_score'][best_index_ridge] * -1

print(f"Fit time: {ridge_fit_time}, Predict time: {ridge_score_time}, Best RMSE: {ridge_RMSE}")

   
```

Или вообще просто:

берем `result = pd.DataFrame(randomized_search.cv_results_)`, уточняем, какие колонки нам нужны, оставляем модели, параметры, время обучения, время предсказания.
       
***
    
Тогда если мы попробуем реализовать пайплайн по подобию пайплайна из спринта по Обучению с учителем, где у нас перебираются несколько моделей и их параметры, то из этого пайплайна сразу получим время обучения и предсказания, а также качество всех моделей по просс-валидации.
    
Все будет компактно, после этого просто допроверим качество на тестовой выборке.
    
Но вполне верным решением будет и делать обучение для каждой модели отдельно.
        
</div>
